<a href="https://colab.research.google.com/github/Nazhan99/Tensorflow_notes/blob/main/07_milestone_project_1_food_vision_big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project 1: Food Vision Big


## Check GPU
check gpu to ensure it compatible with mixed precision training. We nned to access to a Tesla T4 (from google colab) or if we are using our own hardware, our GPU needs a score of 7.0+ (refer: https://developer.nvidia.com/cuda-gpus)

to change Gpu, go to runtime>factory runtime type #this is randomly given

if not get access to 7.0+ GPU, the process can still run but slower.

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-a06d611b-213f-430e-8bef-abc131cf9251)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-02-27 04:49:43--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-02-27 04:49:43 (64.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
#import series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use TensorFlow dataset to download data

> refer: https://www.tensorflow.org/datasets

In [4]:
#get tensorflow datasets
import tensorflow as tf
import tensorflow_datasets as tfds

In [5]:
#list all avalaible datasets
datasets_list = tfds.list_builders() #get all available datasets in TFDS
print("food101" in datasets_list) # is our dataset in the list of tfds datasets?

True


## Exploring the Food101 data from TensorFlow datasets

To become one with our data, we want to find:
* Class names
* The shape of our input data (image tensors)
* The datatype of our input data
* What the labels look like ( are they one-hot label encoded)
* Do the labels match up with the class name

In [6]:
# Load in the data (takes 5-6 minutes in google colab)
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train", "validation"],
                                             shuffle_files = True,
                                             as_supervised = True, #data is return as tupple format (data,labe)
                                             with_info = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteTDKTL7/food101-train.tfrecord


  0%|          | 0/75750 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteTDKTL7/food101-validation.tfrecord


  0%|          | 0/25250 [00:00<?, ? examples/s]

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [7]:
# Features of Food101 from TFDS
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [8]:
#get the class names
class_names = ds_info.features["label"].names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [12]:
#take one sample of the train data
train_one_sample = train_data.take(1) #samples are in format (image_tensor, label)

In [11]:
#what does one sample of our training data look like
train_one_sample

<TakeDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [13]:
#output info about our training smapel
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image datatype: {image.dtype}
  Target class from Food101 (tensor form): {label}
  Class name (str form): {class_names[label.numpy()]}
  """)


  Image shape: (384, 512, 3)
  Image datatype: <dtype: 'uint8'>
  Target class from Food101 (tensor form): 97
  Class name (str form): takoyaki
  
